In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
import pandas as pd
from datetime import date, timedelta 

In [34]:
## Constants: ##
ascertainment = 10
incubation = 4.08
Vne1 = 0.05   # Non-efficacy rate for Pfizer vaccine
Vne2 = 0.059  # Non-efficacy rate for Moderna vaccine
Vne3 = 1      # Non-efficacy rate for no vaccine

# Get some dates we need and convert to strings
today = date.today()
seven_days = str(today - timedelta(days = 7))  
yesterday = str(today - timedelta(days = 1))
day_before = str(today - timedelta(days = 2))

## import data files ##
# 2019 County population data
url = 'https://raw.githubusercontent.com/nericksx/Vaccinated-Gatherings/master/County_pop.csv'
county_df = pd.read_csv(url, header=0)
#county_df.set_index('State', inplace=True)

# Case data from NYT edited down to last 7 days
url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv'
df = pd.read_csv(url)
df.set_index('date', inplace=True) # Set the index as the date col so we can use the truncate function
cases_df = df.truncate(before = seven_days, after = yesterday)

# Make a df of just yesterday's data and one of the day before yesterday's data
yesterday_df = df.truncate(before = yesterday)
day_before_df = df.truncate(before = day_before, after = day_before)

## What state and county are you in?

In [52]:
#state = input("Choose a state: ")
state = 'Oregon'

# Make a df from the rows are just the chosen state
state_df = county_df.loc[county_df['State'] == state]

# Make a list of the values in the 'county' column and print it so the user can reference it
counties_list = state_df['County'].tolist()
#print(counties_list)

#county = input("Choose a county from the list: ")
county = 'Multnomah'

Let's use this info to make a special df that's just our county for the last 7 days.

In [53]:
# Using the county name, create a df that is just one row 
# and access the location containing the FIPS
fips = state_df.loc[state_df['County'] == county, 'FIPS'].iloc[0]

# Use the FIPS to pull just our county from the 7-day cases_df 
target_df = cases_df.loc[cases_df['fips'] == fips]

# Put the index back so we can access the date as a regular column
target_df.reset_index(inplace=True)
target_df

,date,county,state,fips,cases,deaths
0,2021-01-14,Multnomah,Oregon,41051.0,27890,446.0
1,2021-01-15,Multnomah,Oregon,41051.0,28068,457.0
2,2021-01-16,Multnomah,Oregon,41051.0,28237,458.0
3,2021-01-17,Multnomah,Oregon,41051.0,28337,458.0
4,2021-01-18,Multnomah,Oregon,41051.0,28467,463.0
5,2021-01-19,Multnomah,Oregon,41051.0,28586,463.0
6,2021-01-20,Multnomah,Oregon,41051.0,28678,464.0


## Dnp: Predicted Daily Cases

### Lets make a 7 day regression! 

We'll make a list of last 7 dates from our special target_df since it's already such a tidy column

In [54]:
# Convert the date column to a list
dates_list = target_df['date'].tolist()

# Use Pandas to make a stable happy array because it's a datetime
dates = pd.DatetimeIndex(dates_list)

dates

DatetimeIndex(['2021-01-14', '2021-01-15', '2021-01-16', '2021-01-17',
               '2021-01-18', '2021-01-19', '2021-01-20'],
              dtype='datetime64[ns]', freq=None)

Run a linear regression to get the slope

In [39]:
# Put the dates in a Numpy array and reshape to run the regression
x = np.array(dates, dtype='datetime64[D]').reshape((-1,1))

# Daily new cases
y = np.array(['225', '7', '256', '305', '185', '178', '169'])

# Make the regression model
model = LinearRegression().fit(x, y)

#Get the slope and return it as a number (as opposed to an array)
slope = (model.coef_).item(0)
print(slope)

3.6785714285714293


### Use slope to calculate Dnp
(incubation * slope) + yesterday's new cases

In [ ]:
Dnp = (incubation*slope)+169
print(Dnp)

## Irp: Likelihood of a random person being infected in a county
(Dnp * Ascertainment Bias:10 * Incubation period:4.08) / County Size

In [ ]:
Irp = (Dnp * ascertainment * incubation)/812855
print(Irp)

## Let's figure out if you can go to that dinner party!

In [ ]:
n1 = 0  # Number of people with the Pfizer vaccine
n2 = 0  # Number of people with the Moderna vaccine
n3 = 7  # Number of people unvaccinated :-0

pfizer_folks_A = (1-(1-(Vne1 * Irp)) ** n1)
moderna_folks_A = (1-(1-(Vne2 * Irp)) ** n2)
unvacc_folks_A = (1-(1-(Vne3 * Irp)) ** n3) 

pfizer_folks_B = (1-(1-(Vne1 * (1-Irp))) ** n1)
moderna_folks_B = (1-(1-(Vne2 * (1-Irp))) ** n2)
unvacc_folks_B = (1-(1-(Vne3 * (1-Irp))) ** n3)

A = pfizer_folks_A + moderna_folks_A + unvacc_folks_A
B = pfizer_folks_B + moderna_folks_B + unvacc_folks_B

result = (A * B) * 100
chance = round(result, 2)
print('You have a ' + str(chance) + '% ' + 'chance of an exposure occuring at this gathering.')

if chance >= 20:
    print('I wouldn\'t go' )